In [ ]:
import sys
import platform
from importlib import metadata
kernel_name = sys.executable
print(f'✅ Kernel de Jupyter:')
print(f'   - Ruta: {kernel_name}\n')
os_name = platform.system()
os_release = platform.release()
print(f'✅ Sistema Operativo:')
print(f'   - Sistema: {os_name}')
print(f'   - version: {os_release}\n')
print('✅ versions of the Librerías of Qiskit Instaladas:')
try:
    dists = metadata.distributions()
    qiskit_packages = sorted([f"   - {dist.metadata['name']}=={dist.version}" for dist in dists if 'qiskit' in dist.metadata['name']])
    if qiskit_packages:
        for package in qiskit_packages:
            print(package)
    else:
        print('   - No se encontraron librerías of Qiskit instaladas.')
except Exception as e:
    print(f'   - No se pudo verificar the librerías: {e}')
print('\nEntorno verificado! ✨ Ya puedes ejecutar el resto de tu notebook.')

# 🧑‍⚖️ ** Portfolio Optimizacion **

**Objective:** prototipar optimization of portfolio with binary variables and quadratic objective, alineado a the **Deliverables (T1–T5)** of the challenge. **Run All** and guiar the demo with este slide.

**Alignment with the Challenge (T1–T5)**
- **T1 (Formulation):** usamos the formulation *OneOpto* with matching cuadrático and constraints of negocio. Mapa completo PDF→notebook incluido More abajo.  
- **T2 (Conversión quantum):** problema binario → **QUBO** (matching + cardinality) and evaluamos **hinges** for **min/max guardrails per (ℓ,j)** and **RC** (desigualdades).  
- **T3 (Programa quantum):** QAOA (or fallback of muestreo tipo cross‑entropy) sobre the QUBO; métricas se calculan with the **extended cost**.  
- **T4 (Resolver):** se ejecuta end‑to‑end; se reporta energy, selection and **tables of deviations per (ℓ,j)**.  
- **T5 (Validación clásica):** **Exact enumeration** for N≤20 (exact optimum), **gap** QAOA vs. optimum and **prob. of muestreo of the optimum**. Además, **comparative plots** quantum vs. classical hasta **15 qubits**.

**What this adds (innovation/robustness)**
- **Guardrails duros per bisagra**: dos penalizaciones (superior/inferior) per (ℓ,j) for garantizar box without cambiar the QUBO.
- **RC as inequality real** (hinges), evitando solutions ‘in promedio’ fuera of rango.
- **Benchmark “of verdad”**: exact optimum (N≤20) + curves of convergence + scalability (N hasta 15 in demo).

**Demo script (3 steps)** 
1) Execute **Run All**.  
2) Check **optimal line** (si N≤20), **gap** and **optimal sampling probability**.  
3) Show **charts**: quantum vs. classical per iteración; comentar how the **hinges** contienen the exposiciones dentro of the box.

**Reference notes** 
- *Deliverables and criterios of evaluación* (velocidad, optimalidad, scalability).  
- *Formulation matemática*: binary variables, quadratic objective, **guardrails** and **RC** (ver páginas 6/8 and 8/8 of the PDF).  

---
**Key message**: we keep the **QUBO framework** for QAOA and evaluate business metrics with an **extended cost** that impone **min/max box per feature and bucket** and **RC** per desigualdad; we validate against the **exact optimum** and show **convergence** and **scalability**.


## 🔍 Environment diagnostics for **Sampler/QAOA/VQE** (native vs. fallback)
Ejecuta **solo esta cell** si the cuaderno cae in *fallback*. Te devuelve un **reporte completo** of the entorno (Python, pip/conda, kernel, versions of Qiskit and módulos clave), intenta **importar and correr** un `Sampler` minimum and verifica **QAOA/VQE** (rutas modernas/legacy). with esto te puedo decir exactamente what instalar/actualizar.

**what esperar:**
- `sampler_test.import=True` and `sampler_test.run_ok=True`  ⇒ tienes **primitives.Sampler** funcional.
- `QAOA: 'qiskit_algorithms'` or `'qiskit.legacy'` ⇒ hay clase **QAOA** available.
- `optimization=True` ⇒ está **qiskit-optimization** for usar `QuadraticProgram`/`MinimumEigenOptimizer`.

**Siguientes steps:** comparte the bloque of salida of esta cell and te doy the **comando exact** (pip/conda) for dejar QAOA/VQE operativos.

In [ ]:
import sys, os, platform, json, pprint
from importlib import import_module
try:
    from importlib.metadata import version as _ver, packages_distributions as _pkgs
except Exception:
    from importlib_metadata import version as _ver, packages_distributions as _pkgs

def _version(pkg):
    try:
        return _ver(pkg)
    except Exception:
        return None
report = {}
report['python'] = {'version': sys.version.replace('\n', ' '), 'implementation': platform.python_implementation(), 'executable': sys.executable, 'prefix': sys.prefix, 'platform': platform.platform()}
for k in ['CONDA_PREFIX', 'CONDA_DEFAULT_ENV', 'VIRTUAL_ENV', 'PIP_PREFIX']:
    if os.environ.get(k):
        report.setdefault('env', {})[k] = os.environ.get(k)
try:
    import pip
    report['pip'] = {'version': pip.__version__, 'module': getattr(pip, '__file__', None)}
except Exception as e:
    report['pip_error'] = repr(e)
try:
    import conda
    report['conda'] = {'module': getattr(conda, '__file__', None)}
except Exception as e:
    report['conda_info'] = 'conda (módulo) no importable'
try:
    import ipykernel, jupyter_client
    report['ipykernel'] = {'version': ipykernel.__version__}
    report['jupyter_client'] = {'version': jupyter_client.__version__}
except Exception as e:
    report['kernel_info_error'] = repr(e)
pkg_list = ['qiskit', 'qiskit-terra', 'qiskit-aer', 'qiskit-algorithms', 'qiskit-optimization', 'qiskit-ibm-runtime', 'qiskit-nature', 'qiskit-machine-learning']
report['packages'] = {p: _version(p) for p in pkg_list}
try:
    import qiskit
    report['qiskit_meta'] = getattr(qiskit, '__qiskit_version__', None)
except Exception as e:
    report['qiskit_import_error'] = repr(e)
sampler_test = {}
try:
    from qiskit.primitives import Sampler
    sampler_test['import'] = True
    try:
        from qiskit import QuantumCircuit
        qc = QuantumCircuit(1)
        qc.h(0)
        sampler = Sampler()
        try:
            res = sampler.run([qc]).result()
            sampler_test['run_ok'] = True
            sampler_test['result_type'] = type(res).__name__
            sampler_test['summary'] = str(res)[:160] + '...'
        except Exception as e_run:
            sampler_test['run_ok'] = False
            sampler_test['run_error'] = repr(e_run)
    except Exception as e_circ:
        sampler_test['import'] = True
        sampler_test['run_ok'] = False
        sampler_test['run_error'] = f'QuantumCircuit error: {repr(e_circ)}'
except Exception as e_s:
    sampler_test['import'] = False
    sampler_test['error'] = repr(e_s)
report['sampler_test'] = sampler_test
algo = {}
try:
    from qiskit_algorithms.minimum_eigensolvers import QAOA as _QAOA
    algo['QAOA'] = 'qiskit_algorithms'
except Exception as e1:
    try:
        from qiskit.algorithms.minimum_eigensolvers import QAOA as _QAOA
        algo['QAOA'] = 'qiskit.legacy'
    except Exception as e2:
        algo['QAOA'] = f'ERROR: {e1} / {e2}'
try:
    from qiskit_algorithms.minimum_eigensolvers import VQE as _VQE
    algo['VQE'] = 'qiskit_algorithms'
except Exception as e1:
    try:
        from qiskit.algorithms.minimum_eigensolvers import VQE as _VQE
        algo['VQE'] = 'qiskit.legacy'
    except Exception as e2:
        algo['VQE'] = f'ERROR: {e1} / {e2}'
report['algorithms'] = algo
opt = {}
try:
    from qiskit_optimization import QuadraticProgram
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    opt['optimization'] = True
except Exception as e:
    opt['optimization'] = f'ERROR: {repr(e)}'
report['optimization'] = opt
missing = []
if not sampler_test.get('import', False):
    missing.append('qiskit (terra) con primitives.Sampler')
if isinstance(algo.get('QAOA', ''), str) and algo['QAOA'].startswith('ERROR'):
    missing.append('qiskit-algorithms')
if isinstance(algo.get('VQE', ''), str) and algo['VQE'].startswith('ERROR'):
    missing.append('qiskit-algorithms')
if isinstance(opt.get('optimization', True), str):
    missing.append('qiskit-optimization')
if report['packages'].get('qiskit-aer') is None:
    missing.append('qiskit-aer (opcional: simulador)')
recommend = {'pip': 'pip install -U qiskit qiskit-algorithms qiskit-optimization qiskit-aer qiskit-ibm-runtime', 'conda': 'conda install -c conda-forge qiskit qiskit-algorithms qiskit-optimization qiskit-aer'}
report['missing_or_recommended'] = missing
report['install_cmds'] = recommend
print('\n=== QAOA/VQE ENVIRONMENT DIAGNOSTIC ===\n')
pprint.pprint(report, width=140, sort_dicts=False)
print('\nTip: asegúrate of that *pip* corresponde al mismo *Python* of the kernel. Ejecuta dentro of the notebook: `import sys, pip; print(sys.executable, pip.__version__)`\n')

## 🔌 Detect **quantum mode** (QAOA vs. fallback)
Esta cell imprime explícitamente what **mode** se usará. Si **Qiskit/QAOA** están instalados, verás `QAOA (Qiskit + Sampler)`. of lo contrario, se usa un **fallback** of muestreo (*cross‑entropy*).

In [ ]:
MODE = 'fallback'
QAOA_AVAILABLE = False
DETAILS = {}
try:
    import qiskit
    DETAILS['qiskit'] = qiskit.__version__
    try:
        from qiskit.primitives import Sampler
        try:
            import qiskit_algorithms
            from qiskit_algorithms.minimum_eigensolvers import QAOA
            from qiskit_algorithms.optimizers import COBYLA
            QAOA_AVAILABLE = True
            MODE = 'QAOA (Qiskit + Sampler)'
            DETAILS['qiskit_algorithms'] = qiskit_algorithms.__version__
        except Exception:
            from qiskit.algorithms.minimum_eigensolvers import QAOA
            from qiskit.algorithms.optimizers import COBYLA
            QAOA_AVAILABLE = True
            MODE = 'QAOA (qiskit.algorithms legacy)'
            DETAILS['qiskit_algorithms'] = 'legacy'
    except Exception as e_sampler:
        DETAILS['warn_sampler'] = f'Sampler no disponible: {e_sampler}'
except Exception as e_q:
    DETAILS['error'] = f'Qiskit no instalado: {e_q}'
LABEL_Q = f'quantum — {MODE}'
LABEL_C = 'Clásico — búsqueda local'
print('[Quantum Mode]', MODE)
print('Details:', DETAILS)

# Guía ejecutiva and **checklist of the Challenge** (Run All OK)

Este cuaderno conserva **intacto todo the code** and añade cells *Markdown* intermedias that explican: (i) how se cumplen the **5 tareas** of the challenge and (ii) how se mapean the **variables matemáticas of the PDF** a the variables usadas in the notebook.

**Summary of cumplimiento of the 5 tareas:**
- **Tarea 1 — Check the formulation matemática**: Abajo incluimos the **mapa of variables** of the PDF ➜ notebook and References a the láminas clave (*formulation OneOpto* and *formulation binaria for sampling*).
- **Tarea 2 — Convertir a formato compatible with algoritmo quantum**: Se usa the forma **QUBO** of the matching + penalizaciones (cardinality) and se evalúan **hinges** for *guardrails* and **RC** as desigualdades (método of penalización), lista for QAOA.
- **Tarea 3 — Programa of optimization quantum**: the pipeline QAOA (or fallback) ya está integrado; the *Hamiltoniano* se corresponde with the QUBO of matching/cardinality and se reportan métricas sobre the **extended cost**.
- **Tarea 4 — Resolver the problema with the formulation quantum**: Running *Run All* se obtienen bitstrings candidatos, su extended cost and the curva of convergence.
- **Tarea 5 — Validación clásica**: for **N ≤ 20** se hace **Exact Enumeration** of the extended cost; se reporta *exact optimum*, *gap* vs QAOA and **probability of muestreo of the optimum**. También hay utilitarios of métricas and tables.

*(the tareas están definidas in the documento of the challenge; ver References al final of esta guía.)*

## how run (Run All)

1. (Opcional) Ajusta Parameters in the cells of Configuration original si lo deseas.
2. Ejecuta **Run All**. the orden of cells está Ready for that no falle.
3. Check the salidas: extended cost, tables of deviations per *(ℓ,j)*, curva of convergence, and —si **C ≤ 20**— exact optimum, *gap* and optimal sampling probability.

**Importante:** Este cuaderno **no modifica ningún bloque of code existente**. Solo inserta explicaciones *Markdown*.


# Index tracking

notebook of reference for the reto: QUBO of *matching of features per buckets* (version sampling-based) + QAOA (with fallback).
Ejecuta **Run all**; si no hay Qiskit/SciPy, usa fallback without errores.

In [ ]:
import sys, platform, importlib, numpy as np

def _ver(mod):
    try:
        m = importlib.import_module(mod)
        v = getattr(m, '__version__', None)
        return v or 'unknown'
    except Exception as e:
        return f'NOT INSTALLED ({type(e).__name__})'
print('Python :', sys.version.split()[0])
print('OS     :', platform.platform())
print('qiskit :', _ver('qiskit'))
print('qiskit_aer:', _ver('qiskit_aer'))
print('scipy  :', _ver('scipy'))
RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)

# Vanguard Challenge: Quantum Portfolio Optimization

This notebook presents a solution to the Vanguard portfolio optimization challenge. It demonstrates how to use a **Quantum Approximate Optimization Algorithm (QAOA)** to solve a simplified index tracking problem and compares the results against classical benchmarks. 

The solution is structured to address the five key tasks outlined in the challenge document (`01.md`):

1.  **Review the mathematical formulation:** We will define and explain all the input parameters and variables used in the model, mapping them directly to the code.
2.  **Convert to a quantum-compatible formulation:** We will construct a Quadratic Unconstrained Binary Optimization (QUBO) model, which is the standard input format for many quantum optimization algorithms.
3.  **Write a quantum optimization program:** We will implement a QAOA solver using Qiskit.
4.  **Solve the problem:** We will execute the QAOA algorithm to find a solution to the index tracking problem.
5.  **Validate the solution:** We will compare the quantum algorithm's results against both a simple greedy algorithm and a comprehensive exact classical solver to measure performance and solution quality.

In [ ]:
from __future__ import annotations
import numpy as np
Tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'JPM']
TRADING_DAYS = 252

def load_prices_and_stats(tickers, days: int, seed: int=123):
    """Try yfinance; if offline, fall back to synthetic (so Run All never breaks)."""
    try:
        import yfinance as yf
        period = '5y' if days > 730 else '2y' if days > 504 else '1y'
        dfp = yf.download(tickers, period=period, auto_adjust=True, progress=False)['Close']
        dfp = dfp.dropna().tail(days)
        if dfp.shape[0] < min(60, days // 3):
            raise RuntimeError('Insufficient price history. Reduce TRADING_DAYS or adjust tickers.')
        rets = np.log(dfp / dfp.shift(1)).dropna()
        mu = rets.mean().values.astype(float)
        cov = rets.cov().values.astype(float)
        names = list(dfp.columns)
        return (names, mu, cov)
    except Exception as e:
        rng = np.random.default_rng(seed)
        n = len(tickers)
        names = list(tickers)
        mu = rng.normal(0.0005, 0.01, size=n).astype(float)
        A = rng.normal(0, 0.01, size=(n, n))
        cov = A @ A.T
        print('Warning:', type(e).__name__, '-', e)
        print('Using synthetic (mu, cov) to keep Run All robust.')
        return (names, mu, cov)
NAMES, MU, COV = load_prices_and_stats(Tickers, TRADING_DAYS, seed=RANDOM_SEED)
print(f'Universe size: {len(NAMES)} | NAMES = {NAMES}')
print('Shapes -> mu:', MU.shape, 'cov:', COV.shape)

In [ ]:
from __future__ import annotations
import numpy as np
N = len(NAMES)
m = np.zeros(N)
M = np.ones(N)
i = np.full(N, 10.0)
delta = np.ones(N)
XBAR = (m + np.minimum(M, i)) / (2.0 * delta)
vol = np.sqrt(np.clip(np.diag(COV), 1e-08, None))
beta_mu = (MU - MU.mean()) / (MU.std() + 1e-12)
beta_vol = (vol - vol.mean()) / (vol.std() + 1e-12)
rng = np.random.default_rng(42)
beta_style = rng.normal(0, 1, size=N)
BETA = np.column_stack([beta_mu, beta_vol, beta_style]).astype(float)
CHAR_NAMES = ['exp_return_z', 'volatility_z', 'style1']
med = float(np.median(vol))
grp_all = np.arange(N, dtype=int).tolist()
grp_low = np.where(vol <= med)[0].astype(int).tolist()
grp_high = np.where(vol > med)[0].astype(int).tolist()
GROUPS = [('ALL', grp_all), ('LOWVOL', grp_low), ('HIGHVOL', grp_high)]
RHO = np.array([1.0, 0.6, 0.6], dtype=float)
K_TARGET = np.zeros((len(GROUPS), BETA.shape[1]), dtype=float)
for ell, (_, idxs) in enumerate(GROUPS):
    K_TARGET[ell, :] = (BETA[idxs, :] * XBAR[idxs, None]).sum(axis=0)
PRICES = 100.0 + 5.0 * rng.standard_normal(N)
K_MAX = min(6, N)
RC_target_total = float((PRICES * XBAR).sum()) * (K_MAX / max(1, N))
RC_bounds = (0.95 * RC_target_total, 1.05 * RC_target_total)
print('RC bounds ~', RC_bounds)

## Task 1: Reviewing the Mathematical Formulation (Part 1 - Inputs)

The first step in any optimization problem is to define the inputs. The following code loads or synthetically generates the financial data that forms the basis of our portfolio. This corresponds to defining the **set of securities C** from the mathematical formulation (page 8 of `02.pdf`).

- **Universe of Assets:** The list of `Tickers` defines the set of securities $C$ we can choose from. `N = len(NAMES)` is the total number of available assets.
- **Asset Characteristics:** The function `load_prices_and_stats` calculates the mean returns (`MU`) and the covariance matrix (`COV`) of these assets. These are fundamental properties used to derive asset characteristics later on.

In [ ]:
from __future__ import annotations
import numpy as np
from typing import Dict, Tuple, List

def build_qubo_index_tracking(BETA: np.ndarray, GROUPS: List[Tuple[str, List[int]]], K_TARGET: np.ndarray, RHO: np.ndarray, XBAR: np.ndarray, N_max: int, PRICES: np.ndarray, RC_bounds: Tuple[float, float], penalties: dict=None):
    """QUBO for the sampling-based index-tracking objective.


    Objective:

      min  sum_ell rho_ell * sum_j ( sum_{c in K_ell} beta_{c j} * XBAR_c * y_c  -  K_target_{ell j} )^2

    Penalties:

      - Cardinality: P_card * (sum_i y_i - N_max)^2

      - Residual cash flow: with RC* = mean(RC_bounds)

              P_rc * ( sum_i price_i * XBAR_i * y_i  - RC* )^2

    Returns: (h, J, c0) for f(y) = y^T J y + h^T y + c0 with y_i in {0,1}
"""
    N, Jdim = BETA.shape
    if penalties is None:
        penalties = {}
    P_card = float(penalties.get('P_card', 5.0))
    P_rc = float(penalties.get('P_rc', 1.0))
    h: Dict[int, float] = {}
    J: Dict[Tuple[int, int], float] = {}
    c0 = 0.0
    for ell, (_, idxs) in enumerate(GROUPS):
        rho = float(RHO[ell])
        for j in range(Jdim):
            k_t = float(K_TARGET[ell, j])
            c0 += rho * k_t ** 2
            for ii_pos, i in enumerate(idxs):
                bi = float(BETA[i, j]) * float(XBAR[i])
                h[i] = h.get(i, 0.0) + rho * bi ** 2
                for j_pos in range(ii_pos + 1, len(idxs)):
                    k = idxs[j_pos]
                    bk = float(BETA[k, j]) * float(XBAR[k])
                    J[min(i, k), max(i, k)] = J.get((min(i, k), max(i, k)), 0.0) + 2.0 * rho * bi * bk
            for i in idxs:
                bi = float(BETA[i, j]) * float(XBAR[i])
                h[i] = h.get(i, 0.0) - 2.0 * rho * k_t * bi
    c0 += P_card * N_max ** 2
    for i in range(N):
        h[i] = h.get(i, 0.0) + P_card * (1.0 - 2.0 * N_max)
    for i in range(N):
        for k in range(i + 1, N):
            J[i, k] = J.get((i, k), 0.0) + 2.0 * P_card
    RC_star = 0.5 * (float(RC_bounds[0]) + float(RC_bounds[1]))
    c0 += P_rc * RC_star ** 2
    pbar = PRICES * XBAR
    for i in range(N):
        h[i] = h.get(i, 0.0) + P_rc * (pbar[i] ** 2 - 2.0 * RC_star * pbar[i])
    for i in range(N):
        for k in range(i + 1, N):
            J[i, k] = J.get((i, k), 0.0) + 2.0 * P_rc * pbar[i] * pbar[k]
    return (h, J, float(c0))

def energy_qubo_bitstring(bitstr: str, h: Dict[int, float], J: Dict[Tuple[int, int], float], c0: float, n: int) -> float:
    s = bitstr.rjust(n, '0')
    x = np.array([1 if s[-1 - i] == '1' else 0 for i in range(n)], dtype=int)
    val = c0
    for i, hi in h.items():
        val += hi * x[i]
    for (i, j), Jij in J.items():
        ii, jj = (i, j) if i < j else (j, i)
        val += Jij * x[ii] * x[jj]
    return float(val)

def decode_selection_from_bitstring(bitstr: str, names: list[str]) -> list[str]:
    s = bitstr.rjust(len(names), '0')
    return [names[i] for i in range(len(names)) if s[-1 - i] == '1']
PENALTIES = {'P_card': 5.0, 'P_rc': 1.0}
QUBO_H, QUBO_J, QUBO_C0 = build_qubo_index_tracking(BETA=BETA, GROUPS=GROUPS, K_TARGET=K_TARGET, RHO=RHO, XBAR=XBAR, N_max=K_MAX, PRICES=PRICES, RC_bounds=RC_bounds, penalties=PENALTIES)
print(f'QUBO built: |h|={len(QUBO_H)} |J|={len(QUBO_J)} |c0|={QUBO_C0:.4f}')

--- 
## ⚙️ solution Ideal (Entorno Funcional) — Opción 1: VQE
the siguiente cell muestra how resolver the problema of optimization utilizando the **Variational Quantum Eigensolver (VQE)**, un algoritmo híbrido muy popular.

**Lógica:**
1.  **Formulation of the Problema:** Se utiliza `PortfolioOptimization` of `qiskit_optimization` for definir the problema with the retornos, covarianzas, risk aversion and presupuesto.
2.  **Conversión a Hamiltoniano:** the problema se convierte a un Hamiltoniano of Ising. VQE funciona encontrando the estado of energy mínima (estado fundamental) of este Hamiltoniano, that corresponde a the solution óptima of nuestra portfolio.
3.  **execution of VQE:** Se configura un `Estimator` for medir the Results, un circuito parametrizado (`ansatz`, in este caso `TwoLocal`), and un optimizador classical (`SPSA`). VQE itera, ajustando the Parameters of the ansatz with the optimizador classical for minimizar the energy medida per the `Estimator`.
4.  **Interpretación:** the result of VQE se decodifica of nuevo a una selection of activos.

*(Esta cell solo se ejecutará si the variable `QAOA_AVAILABLE` es `True`, indicando un entorno Qiskit moderno and funcional.)*

In [ ]:
if QAOA_AVAILABLE:
    print('⚙️ Running the solution VQE (entorno funcional detectado)...\n')
    from qiskit.primitives import Estimator
    from qiskit_algorithms import VQE
    from qiskit_algorithms.optimizers import SPSA
    from qiskit.circuit.library import TwoLocal
    from qiskit_optimization.applications import PortfolioOptimization
    from qiskit_optimization.converters import QuadraticProgramToQubo
    from qiskit.quantum_info import PauliSumOp
    q_risk_factor = 0.5
    budget = K_MAX
    portfolio = PortfolioOptimization(expected_returns=MU, covariances=COV, risk_factor=q_risk_factor, budget=budget)
    qp = portfolio.to_quadratic_program()
    converter = QuadraticProgramToQubo()
    qubo = converter.convert(qp)
    hamiltonian, offset = qubo.to_ising()
    hamiltonian_op = PauliSumOp.from_list(hamiltonian.primitive.to_list())
    print('--- Modelo of optimization for VQE Creado ---')
    print(f'Número of activos (qubits): {N}')
    print(f'Presupuesto: {budget}\n')
    estimator = Estimator()
    ansatz = TwoLocal(num_qubits=N, rotation_blocks='ry', entanglement_blocks='cz')
    optimizer = SPSA(maxiter=100)
    vqe_solver = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer)
    result_vqe = vqe_solver.compute_minimum_eigenvalue(hamiltonian_op)
    selection_vqe = portfolio.interpret(result_vqe)
    print('--- VQE Ejecutado ---')
    print(f'solution VQE encontrada: {selection_vqe}')
    print(f'Activos seleccionados: {[NAMES[i] for i, x in enumerate(selection_vqe) if x == 1]}\n')
else:
    print('Skipping VQE: Entorno funcional no detectado (QAOA_AVAILABLE=False).')

---
## ⚙️ solution Ideal (Entorno Funcional) — Opción 2: QAOA with Optimizador
Esta cell muestra the forma moderna e idiomática of resolver the problema with **QAOA**, utilizando the abstracciones of alto nivel of Qiskit.

**Lógica:**
1.  **Formulation of the Problema:** Reutilizamos the `QuadraticProgram` (`qp`) definido in the sección of VQE. Este objeto contiene toda the información matemática of the problema of portfolio.
2.  **Configuration of QAOA:** Creamos una instancia of `qiskit_algorithms.QAOA`, especificando un `Sampler` (for muestrear the Results of the circuito) and un optimizador classical (`COBYLA`).
3.  **Optimizador of Alto Nivel:** in lugar of manejar the bucle of optimization manualmente, usamos the `MinimumEigenOptimizer`. Este es un "envoltorio" inteligente that sabe how usar un algoritmo quantum (as QAOA) for resolver un `QuadraticProgram`.
4.  **Resolución e Interpretación:** Simplemente llamamos a `optimizer.solve(qp)`. the result es un objeto that contiene directamente the solution óptima (`.x`) and the value of the function objective (`.fval`).

*(Esta cell también requiere that `QAOA_AVAILABLE` sea `True`.)*

In [ ]:
if QAOA_AVAILABLE:
    print('⚙️ Running the solution QAOA with MinimumEigenOptimizer (entorno funcional detectado)...\n')
    from qiskit.primitives import Sampler
    from qiskit_algorithms.minimum_eigensolvers import QAOA
    from qiskit_algorithms.optimizers import COBYLA
    from qiskit_optimization.applications import PortfolioOptimization
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    if 'qp' not in locals():
        q_risk_factor = 0.5
        budget = K_MAX
        portfolio = PortfolioOptimization(expected_returns=MU, covariances=COV, risk_factor=q_risk_factor, budget=budget)
        qp = portfolio.to_quadratic_program()
        print('--- Modelo of optimization for QAOA Creado ---')
        print(f'Número of activos (qubits): {N}')
        print(f'Presupuesto: {budget}\n')
    sampler = Sampler()
    optimizer_qaoa = COBYLA(maxiter=100)
    qaoa = QAOA(sampler=sampler, optimizer=optimizer_qaoa, reps=2)
    qaoa_optimizer = MinimumEigenOptimizer(qaoa)
    result_qaoa = qaoa_optimizer.solve(qp)
    print('--- QAOA Ejecutado ---')
    print(f'solution QAOA encontrada: {result_qaoa.x}')
    print(f'Valor objective: {result_qaoa.fval}')
    print(f'Activos seleccionados: {[NAMES[i] for i, x in enumerate(result_qaoa.x) if x == 1]}\n')
else:
    print('Skipping QAOA: Entorno funcional no detectado (QAOA_AVAILABLE=False).')

---
## ⚪ solution of Fallback (Entorno no funcional)
Dado that no se ha detectado un entorno Qiskit completamente funcional (`QAOA_AVAILABLE=False`), the siguiente cell utilizará un método of **fallback classical** for generar una solution.

**Lógica:**
- **Qiskit Manual (si está parcialmente available):** Intenta construir and optimizar un circuito QAOA of bajo nivel. Esto puede funcionar in algunas versions antiguas of Qiskit.
- **Gibbs-like Sampling (si Qiskit falla):** Si lo anterior falla, recurre a un método puramente classical. Primero encuentra una solution "buena" with un algoritmo codicioso and luego muestrea solutions in su vecindad, ponderadas per una distribución tipo Gibbs/Boltzmann. Esto asegura that the notebook siempre produzca una salida, incluso without un entorno quantum.

In [ ]:
from __future__ import annotations
import numpy as np
from typing import Dict, Tuple
try:
    from qiskit import QuantumCircuit, transpile
    from qiskit_aer import AerSimulator
    from qiskit.circuit import ParameterVector
    HAVE_QISKIT = True
except Exception as _e:
    HAVE_QISKIT = False

def qaoa_counts_or_fallback(h: Dict[int, float], J: Dict[Tuple[int, int], float], c0: float, n: int, p: int=2, shots: int=1024, seed: int=123):
    rng = np.random.default_rng(seed)
    if HAVE_QISKIT:

        def build_qaoa_circuit(N, p, Hlin, Jqq):
            gammas = ParameterVector('gamma', p)
            betas = ParameterVector('beta', p)
            qc = QuantumCircuit(N, name='QAOA')
            for q in range(N):
                qc.h(q)
            for layer in range(p):
                g = gammas[layer]
                b = betas[layer]
                for i, w in Hlin.items():
                    if abs(w) > 1e-15:
                        qc.rz(2.0 * g * w, i)
                for (i, j), w in Jqq.items():
                    if i != j and abs(w) > 1e-15:
                        ii, jj = (i, j) if i < j else (j, i)
                        qc.cx(ii, jj)
                        qc.rz(2.0 * g * w, jj)
                        qc.cx(ii, jj)
                for q in range(N):
                    qc.rx(2.0 * b, q)
            qc.measure_all()
            return (qc, gammas, betas)

        def energy_from_counts(counts: Dict[str, int]):
            total = float(sum(counts.values())) or 1.0
            E = 0.0
            for bitstr, c in counts.items():
                E += c / total * energy_qubo_bitstring(bitstr, h, J, c0, n)
            return float(E)
        qc_templ, gvec, bvec = build_qaoa_circuit(n, p, h, J)
        backend = AerSimulator(method='matrix_product_state', seed_simulator=int(seed))

        def evaluate(theta):
            bind = {**{gvec[k]: float(theta[k]) for k in range(p)}, **{bvec[k]: float(theta[p + k]) for k in range(p)}}
            qc_bound = qc_templ.assign_parameters(bind, inplace=False)
            qc_run = transpile(qc_bound, backend=backend, seed_transpiler=int(seed))
            result = backend.run(qc_run, shots=int(shots), seed_simulator=int(seed)).result()
            return energy_from_counts(result.get_counts())
        theta0 = np.concatenate([np.full(p, 0.8, dtype=float), np.full(p, 0.4, dtype=float)])
        try:
            from scipy.optimize import minimize
            res = minimize(lambda th: evaluate(np.array(th, dtype=float)), theta0, method='COBYLA', options={'maxiter': 80, 'rhobeg': 0.5, 'tol': 0.001})
            theta_opt = np.array(res.x, dtype=float)
        except Exception as e:
            grid = np.linspace(0.2, 1.2, 5)
            best = (1000000000.0, theta0)
            for g in grid:
                for b in grid:
                    th = np.concatenate([np.full(p, g), np.full(p, b)])
                    val = evaluate(th)
                    if val < best[0]:
                        best = (val, th)
            theta_opt = best[1]
        bind_opt = {**{gvec[k]: float(theta_opt[k]) for k in range(p)}, **{bvec[k]: float(theta_opt[p + k]) for k in range(p)}}
        qc_bound = qc_templ.assign_parameters(bind_opt, inplace=False)
        backend = AerSimulator(method='matrix_product_state', seed_simulator=int(seed))
        counts = backend.run(transpile(qc_bound, backend=backend, seed_transpiler=int(seed)), shots=int(shots), seed_simulator=int(seed)).result().get_counts()
        E_mean = energy_from_counts(counts)
        return (counts, float(E_mean))
    else:

        def greedy_bitstring(kmax: int):
            chosen = []
            available = list(range(n))
            s = '0' * n
            for _ in range(min(kmax, n)):
                best_improve = (1e+18, None, None)
                for i in available:
                    bs = list(s)
                    bs[-1 - i] = '1'
                    bs = ''.join(bs)
                    E = energy_qubo_bitstring(bs, h, J, c0, n)
                    if E < best_improve[0]:
                        best_improve = (E, i, bs)
                if best_improve[1] is None:
                    break
                _, i, s = best_improve
                chosen.append(i)
                available.remove(i)
            return s
        best = greedy_bitstring(kmax=min(6, n))
        Emin = energy_qubo_bitstring(best, h, J, c0, n)
        S = {best: Emin}
        for _ in range(500):
            bs = list(best)
            for __ in range(np.random.randint(1, 4)):
                pos = np.random.randint(0, n)
                bs[-1 - pos] = '1' if bs[-1 - pos] == '0' else '0'
            bs = ''.join(bs)
            S[bs] = energy_qubo_bitstring(bs, h, J, c0, n)
        energies = np.array(list(S.values()), dtype=float)
        T = max(1.0, np.std(energies))
        weights = np.exp(-(energies - energies.min()) / (T + 1e-09))
        probs = weights / weights.sum()
        keys = list(S.keys())
        draws = np.random.choice(len(keys), size=shots, p=probs)
        counts = {}
        for d in draws:
            k = keys[d]
            counts[k] = counts.get(k, 0) + 1
        E_mean = float(np.average(energies, weights=probs))
        return (counts, E_mean)
P, SHOTS = (2, 1024)
counts_opt, E_qaoa = qaoa_counts_or_fallback(QUBO_H, QUBO_J, QUBO_C0, n=N, p=P, shots=SHOTS, seed=RANDOM_SEED)
best_bitstr = max(counts_opt.items(), key=lambda kv: kv[1])[0]
seleccion_qaoa = decode_selection_from_bitstring(best_bitstr, NAMES)
print('Best bitstring:', best_bitstr, '| selection:', seleccion_qaoa)
print('E_mean (QAOA/fallback):', round(E_qaoa, 6))

## Task 2: Convert to a Quantum-Compatible Formulation (QUBO)

Quantum annealers and QAOA solve problems formulated as a **QUBO**. This task involves converting our constrained optimization problem into this unconstrained format. The goal is to minimize an energy function of the form:

$$ E(and) = \sum_{i,j} y_i J_{ij} y_j + \sum_i h_i y_i + c_0 $$

The function `build_qubo_index_tracking` performs this conversion:

1.  **Objective Function:** The original quadratic objective, $\min \sum_{\ell,j} \rho_\ell (\sum_{c \in K_\ell} \beta_{c,j} \bar{x}_c y_c - K_{\ell,j}^{\text{target}})^2$, is expanded into its quadratic ($y_i y_j$), linear ($y_i$), and constant terms. These terms are added to the QUBO coefficients `J`, `h`, and `c0`.

2.  **Constraints as Penalties:** The linear constraints (cardinality and residual cash) are converted into quadratic penalty terms and added to the energy function. For example, the cardinality constraint $\sum y_c \le K_{\text{MAX}}$ is encoded as a penalty $P_{\text{card}} (\sum y_c - K_{\text{MAX}})^2$. A solution that violates the constraint will have a higher energy, making it less likely to be chosen.

By combining the original objective and the constraint penalties into a single energy function, we get a QUBO that a quantum algorithm can solve.

In [ ]:
from __future__ import annotations
import numpy as np

def greedy_same_cost(h, J, c0, n, kmax):
    selected = []
    available = list(range(n))
    s = ['0'] * n
    for _ in range(min(kmax, n)):
        best = (1e+18, None)
        for i in available:
            s_try = s.copy()
            s_try[-1 - i] = '1'
            E = energy_qubo_bitstring(''.join(s_try), h, J, c0, n)
            if E < best[0]:
                best = (E, i)
        if best[1] is None:
            break
        s[-1 - best[1]] = '1'
        selected.append(best[1])
        available.remove(best[1])
    bitstr = ''.join(s)
    return (bitstr, selected, energy_qubo_bitstring(bitstr, h, J, c0, n))
x_class, sel_idx, E_class = greedy_same_cost(QUBO_H, QUBO_J, QUBO_C0, N, K_MAX)
seleccion_classic = [NAMES[i] for i in sel_idx]
print('Greedy baseline: ', seleccion_classic, '| bitstring:', x_class, '| E≈', round(E_class, 6))
print('ΔE (QAOA - Greedy) =', round(E_qaoa - E_class, 6))

## Task 3 & 4: Write and Execute a Quantum Optimization Program

This section addresses the core quantum part of the challenge. The function `qaoa_counts_or_fallback` implements the **Quantum Approximate Optimization Algorithm (QAOA)**, a hybrid quantum-classical algorithm well-suited for near-term quantum computers.

The process is as follows:
1.  **Build QAOA Circuit (`build_qaoa_circuit`):** A parameterized quantum circuit is constructed based on the QUBO problem. It consists of layers of two types of operators:
    - A **phase-separation** operator, which encodes the QUBO objective function.
    - A **mixing** operator, which allows the algorithm to explore the solution space.
2.  **Classical Optimization Loop:** A classical optimizer (like SciPy's `COBYLA`) is used to find the optimal parameters (angles $\gamma$ and $\beta$) for the QAOA circuit. The optimizer's goal is to find parameters that minimize the expected energy of the circuit's output.
3.  **Execution and Measurement:** Once the optimal parameters are found, the final parameterized circuit is run on a quantum simulator (`qiskit_aer.AerSimulator`) thousands of times (`shots`). 
4.  **Get Results:** The output is a dictionary of `counts`, where each key is a bitstring (a potential solution, e.g., `'01011101'`) and the value is the number of times that solution was measured. The bitstring with the highest count is our proposed optimal solution.

The code includes a classical fallback method to ensure it runs even without Qiskit, which performs Gibbs-like sampling around a greedy solution.

In [ ]:
from __future__ import annotations
TOP_K = 10
items = sorted(counts_opt.items(), key=lambda x: -x[1])[:TOP_K]
total = float(sum(counts_opt.values())) or 1.0
print(f'Top-{TOP_K} bitstrings (approximate probabilities):')
for b, c in items:
    p = c / total
    print(f'  {b}  |  p≈{p:.4f}  |  E={energy_qubo_bitstring(b, QUBO_H, QUBO_J, QUBO_C0, N):.6f}')

## Task 5: Validate Your Solution (Part 1 - Greedy Baseline)

To understand how well our QAOA solver performs, we must compare it to a classical method. This cell implements a **simple greedy algorithm** as a first benchmark.

The `greedy_same_cost` function builds a solution iteratively: at each step, it adds the single best asset (the one that results in the lowest immediate cost) to the portfolio, until the maximum number of assets `kmax` is reached.

This is a fast but often suboptimal approach. We compare the final energy of the greedy solution (`E_class`) with the average energy from QAOA (`E_qaoa`). A large difference can indicate the complexity of the problem and the potential for quantum algorithms to find better solutions that are not obvious to a greedy search.

In [ ]:
import json, time
artefact = {'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'), 'model': 'index_tracking_by_characteristics', 'universe': NAMES, 'K_MAX': int(K_MAX), 'penalties': {'P_card': 5.0, 'P_rc': 1.0}, 'bucket_weights_rho': list(map(float, RHO)), 'qaoa_or_fallback': 'Qiskit' if 'QuantumCircuit' in globals() else 'Gibbs-fallback', 'qaoa': {'best_bitstring': max(counts_opt.items(), key=lambda kv: kv[1])[0], 'selection': seleccion_qaoa, 'E_mean': float(E_qaoa)}, 'baseline_greedy': {'bitstring': x_class, 'selection': seleccion_classic, 'E': float(E_class)}}
print(json.dumps(artefact, indent=2)[:1000] + ('...' if len(json.dumps(artefact)) > 1000 else ''))

## Task 5: Validate Your Solution (Part 2 - Solution Analysis)

One of the potential advantages of sampling-based quantum algorithms like QAOA is their ability to find a diverse set of high-quality solutions, not just a single one. This is crucial for portfolio managers who may want to evaluate several near-optimal choices.

This cell inspects the `counts` dictionary from the QAOA run to show the **top 10 most frequently sampled solutions**. For each solution (bitstring), we display:

- The approximate probability of observing it ($p \approx \text{count} / \text{total shots}$).
- Its corresponding QUBO energy (`E`).

This analysis provides insight into the solution landscape. If there are many different bitstrings with low energies, it suggests that multiple distinct portfolios could meet the investment objectives.

# Extensión: Guardrails min/max per feature and bucket (bisagra), RC per desigualdad and **Exact Enumeration** (Entrega 5)

**what añade esta sección:**
1) Penalizaciones **tipo bisagra** per cada \((\ell,j)\) for respetar the **box** \([K^{low}_{\ell j},K^{up}_{\ell j}]\) without cambiar the marco QUBO.
2) **Residual Cash (RC)** as desigualdades reales with hinges: \([RC_{min},RC_{max}]\).
3) **Benchmark classical of verdad**: **Exact Enumeration** for \(N\le 20\), with reporte of optimum, *gap* vs. QAOA/fallback and probability of muestreo of the optimum.
4) Utilidades for **tables of deviations per bucket/feature** and **curva of convergence**.

> Reference of the model simplificado (binario): ver *slide 8* of the PDF (`02.pdf`) with the límites min/max per feature and bucket and the function of matching cuadrática. Entregables of validación (Entrega 5): ver `01.md`. 


## Formulation (hinges)

Sea 
\(
s_{\ell j}(and)=\sum_{c\in K_\ell}\beta_{cj}\,\bar x_c\,y_c
\).
the penalizaciones per **box** of \((\ell,j)\) se añaden al coste as:
\[
P^{+}_{\ell j}\,[\max(0,\,s_{\ell j}(and)-K^{up}_{\ell j})]^2
\;+\;
P^{-}_{\ell j}\,[\max(0,\,K^{low}_{\ell j}-s_{\ell j}(and))]^2.
\]

**RC** per desigualdad:
\[
P^{RC}_+\,[\max(0,\,RC(and)-RC_{max})]^2
\;+\;
P^{RC}_-\,[\max(0,\,RC_{min}-RC(and))]^2,\qquad
RC(and)=\sum_c p_c\,\bar x_c\,y_c.
\]

the **extended energy** that reportamos for métricas es:
\[
C(and)=\underbrace{\sum_{\ell}\rho_\ell\sum_j(s_{\ell j}(and)-K^{target}_{\ell j})^2}_{\text{matching}}
+P_{card}\,(\sum_c y_c-N_{max})^2
+\text{hinges(guardrails)}
+\text{hinges(RC)}.
\]


## Mapa of variables of the **PDF → notebook**

**Conjunto and dimensiones**
- Universo \(C\) (bonos/activos) → tamaño `C` (dimensión of `and`, `xbar`, `prices`).
- *Buckets* of riesgo \(\ell=1..L\) → `K_sets` (lista of índices per bucket; construimos the tensor `A`).
- Características \(j=1..J\) → dimensión `J` (columns of `beta`).

**variables and Parameters of the PDF** *(ver láminas of formulation)*:
- Binaria \(y_c\in\{0,1\}\) (si the bono entra) → vector `and`.
- Cantidad \(x_c\) (no variable in the version binaria; se fija al **promedio permitido** si \(y_c=1\)) → `xbar[c]`.
- Precio \(p_c\) → `prices[c]`.
- Aportes per feature \(\beta_{cj}\) → matrix `beta[c,j]`; se combina with `xbar` in `A[ℓ,j,c]`.
- Contribución agregada per bucket/feature \(s_{\ell j}(and)=\sum_{c\in K_\ell}\beta_{cj}x_c y_c\) → function `s_lj(and,A)`.
- *Targets* \(K^{target}_{\ell j}\) → matrix `K_target[L,J]`.
- *Guardrails* per box \([K^{low}_{\ell j}, K^{up}_{\ell j}]\) → `K_low[L,J]`, `K_up[L,J]`.
- Pesos per bucket \(\rho_\ell\) → `rho[ℓ]`.
- **Residual Cash (RC)** with límites \([RC_{min}, RC_{max}]\) → escalares `RC_min`, `RC_max`.
- Máximo of bonos \(N\) (cardinality) → `card_target`/`Nmax` and peso `P_card`.

**Objective and constraints**
- *Matching cuadrático* per feature/bucket alrededor of `K_target` (function `match_fn(and)`).
- **Box per (ℓ,j)** with **hinges**: penalizamos violaciones with `penalty_guardrails(...)`.
- **RC** as desigualdad with bisagra: `penalty_rc_hinges(...)`.
- **Coste extendido** = matching + cardinality + hinges(guardrails) + hinges(RC): `extended_cost(...)`.

*References visuales: OneOpto (formulation completa) in the **pág. 6/8**; version **binaria** for sampling in the **pág. 8/8** of the PDF.*


In [ ]:
import numpy as np
from typing import Dict, Any, Tuple

def build_A_tensor(beta: np.ndarray, xbar: np.ndarray, K_sets: list) -> np.ndarray:
    """
    Construye A[ell, j, c] = beta[c, j] * xbar[c] si c in K_ell, 0 in otro caso.
    beta: (C, J)
    xbar: (C,)
    K_sets: lista of listas with índices c pertenecientes a cada bucket ell.
    """
    C, J = beta.shape
    L = len(K_sets)
    A = np.zeros((L, J, C), dtype=float)
    for ell, K_ell in enumerate(K_sets):
        K_ell = np.asarray(K_ell, dtype=int)
        if K_ell.size == 0:
            continue
        A[ell, :, K_ell] = beta[K_ell, :] * xbar[K_ell, None]
    return A

def s_lj(y: np.ndarray, A: np.ndarray) -> np.ndarray:
    """
    Devuelve s[ell, j] = sum_c A[ell, j, c] * y[c].
    y: (C,) binaria.
    A: (L, J, C).
    """
    return np.tensordot(A, y, axes=([2], [0]))

def _hinge_pos(x: np.ndarray) -> np.ndarray:
    return np.maximum(0.0, x)

def penalty_guardrails(y: np.ndarray, A: np.ndarray, K_low: np.ndarray, K_up: np.ndarray, P_plus, P_minus) -> Tuple[float, Dict[str, Any]]:
    """
    Penalización por bisagras for the caja [K_low, K_up] por (ell, j).
    K_low, K_up: (L, J). P_plus, P_minus: escalar or (L, J).
    """
    S = s_lj(y, A)
    P_plus = np.broadcast_to(np.array(P_plus, dtype=float), S.shape)
    P_minus = np.broadcast_to(np.array(P_minus, dtype=float), S.shape)
    v_up = _hinge_pos(S - K_up)
    v_low = _hinge_pos(K_low - S)
    cost = np.sum(P_plus * v_up ** 2 + P_minus * v_low ** 2)
    info = {'S': S, 'vio_up': v_up, 'vio_low': v_low}
    return (float(cost), info)

def compute_RC(y: np.ndarray, prices: np.ndarray, xbar: np.ndarray) -> float:
    return float(np.dot(prices * xbar, y))

def penalty_rc_hinges(y: np.ndarray, prices: np.ndarray, xbar: np.ndarray, RC_min: float, RC_max: float, P_plus: float, P_minus: float) -> Tuple[float, Dict[str, Any]]:
    RC = compute_RC(y, prices, xbar)
    up = max(0.0, RC - RC_max)
    low = max(0.0, RC_min - RC)
    cost = P_plus * up ** 2 + P_minus * low ** 2
    info = {'RC': RC, 'vio_up': up, 'vio_low': low}
    return (float(cost), info)

## Task 5: Validate Your Solution (Part 3 - Final Report)

This final cell for the first part of the analysis compiles all the key results into a structured JSON object. This serves as a formal record of the experiment, summarizing:

- The problem setup (`universe`, `K_MAX`, `penalties`).
- The best solution found by QAOA and its mean energy.
- The solution found by the greedy classical baseline and its energy.

This encapsulates the comparison of solution quality and provides a clear, machine-readable output of the project's findings.

## Tareas 2 and 4 — **Conversión a formato quantum** and **resolución**

- the conversión a **formulation compatible with QAOA** se hace vía **penalización**: the matching and the cardinality están in the QUBO; the hinges of guardrails and RC se **evalúan** in the extended cost (`extended_cost(...)`).
- the pipeline QAOA/fallback produce bitstrings `and`; cada `and` se puntúa with the **extended cost** for métricas of negocio.
- Si quieres that the hinges **entren al Hamiltoniano** 1:1, se puede compilar a QUBO with ancillas (no alteramos the code aquí; es opcional).


### Box residual (RC) as desigualdad real

Se penalizan the violaciones a \([RC_{min}, RC_{max}]\) mediante hinges:
\[ P^{RC}_+\,[\max(0, RC(and)-RC_{max})]^2 + P^{RC}_-\,[\max(0, RC_{min}-RC(and))]^2,\quad RC(and)=\sum_c p_c\,x_c\,y_c. \]
Implementado in `penalty_rc_hinges(...)`. Evita solutions ‘in promedio’ fuera of límites.


### Guardrails min/max per feature and bucket (bisagra)

for cada par \((\ell,j)\) se añade:
\[ P^{+}_{\ell j}\,[\max(0, s_{\ell j}(and)-K^{up}_{\ell j})]^2 + P^{-}_{\ell j}\,[\max(0, K^{low}_{\ell j}-s_{\ell j}(and))]^2 \]
where \(s_{\ell j}(and)=\sum_{c\in K_\ell}\beta_{cj}\,x_c\,y_c\). in the cuaderno se implementa with `build_A_tensor(...)`, `s_lj(...)` and `penalty_guardrails(...)`. Así, the solution **respeta the box** of cada feature without tocar the marco QUBO.


In [ ]:
def extended_cost(y: np.ndarray, match_fn, A: np.ndarray, K_low: np.ndarray, K_up: np.ndarray, Pp_guard, Pm_guard, prices: np.ndarray, xbar: np.ndarray, RC_min: float, RC_max: float, Pp_RC: float, Pm_RC: float, card_target: int=None, P_card: float=0.0):
    """
    Devuelve coste total C(and) and desglose.
    - match_fn(and): mantiene tu matching (without cambiar QUBO).
    - bisagras guardrails and RC añaden caja dura in the evaluación.
    """
    c_match = float(match_fn(y))
    c_guard, d_guard = penalty_guardrails(y, A, K_low, K_up, Pp_guard, Pm_guard)
    c_rc, d_rc = penalty_rc_hinges(y, prices, xbar, RC_min, RC_max, Pp_RC, Pm_RC)
    c_card = float(P_card * (np.sum(y) - card_target) ** 2) if card_target is not None and P_card > 0 else 0.0
    total = c_match + c_guard + c_rc + c_card
    detail = {'match': c_match, 'guardrails': d_guard, 'rc': d_rc, 'card': c_card, 'total': total}
    return (total, detail)

## Tarea 5 — **Validación clásica** (benchmark ‘of verdad’)

- **Exact enumeration** for **N ≤ 20** with `enumerate_exact_cost(...)`: obtiene the **exact optimum** of the extended cost.
- Reporta **gap** vs QAOA and **probability of muestreo of the optimum** (si hay `counts`).
- También se generan **tables of deviations** per bucket/feature with `deviations_table(...)`.


In [ ]:
from math import inf
import numpy as np

def _states_to_bits(states: np.ndarray, N: int) -> np.ndarray:
    return (states[:, None] >> np.arange(N, dtype=np.uint32) & 1).astype(np.int8)

def enumerate_exact_cost(N: int, eval_cost_fn, chunk_size: int=16384, return_all: bool=False):
    """
    Recorre the 2^N bitstrings (in bloques) and devuelve the óptimo.
    """
    total_states = 1 << N
    best_cost, best_y, best_idx = (inf, None, -1)
    all_costs = [] if return_all else None
    for start in range(0, total_states, chunk_size):
        end = min(start + chunk_size, total_states)
        states = np.arange(start, end, dtype=np.uint32)
        Y = _states_to_bits(states, N)
        for i in range(Y.shape[0]):
            y = Y[i]
            out = eval_cost_fn(y)
            c = out[0] if isinstance(out, tuple) else float(out)
            if return_all:
                all_costs.append(c)
            if c < best_cost:
                best_cost = c
                best_y = y.copy()
                best_idx = start + i
    meta = {'best_state_index': int(best_idx), 'total_states': int(total_states)}
    return (float(best_cost), best_y, meta, np.array(all_costs, dtype=float) if return_all else None)

## Task 5: Validate Your Solution (Part 4 - Advanced Validation Framework)

While the first section provided a complete solution, this extension offers a more rigorous and powerful framework for validation, directly addressing the challenge's request for comparison against a **benchmark classical solution** in terms of **cost function** and **optimality**.

This section introduces two critical enhancements:

1.  **More Realistic Cost Function (`extended_cost`):** We move beyond simple quadratic penalties. Here, constraints are handled with **hinge loss functions**. A penalty is applied only if a constraint is violated (e.g., if $\sum_c y_c > N_{\max}$), which is a more accurate way to model hard constraints or "guardrails" as described on page 6 of the PDF. This allows for a more precise evaluation of solution quality.

2.  **Exact Classical Solver (`enumerate_exact_cost`):** For small to medium-sized problems ($N \le 20$), it's possible to find the *true global optimum* by simply testing every single possible combination. This method, while slow, provides the perfect "ground truth" benchmark. We can compare the QAOA solution's cost directly to the best possible cost, giving us a precise measure of its optimality (the **optimality gap**).

## Métricas and reportes

- `report_gap(...)`: *gap absoluto/relativo* and *p(muestrear optimum)*.
- `deviations_table(...)`: muestra \(s_{\ell j}(and)\), límites `K_low`/`K_up` (and `K_target` si se provee), and violaciones.
- `plot_convergence(...)`: energy vs iteración for the **curva of convergence**.


In [ ]:
import numpy as np
import pandas as pd
from typing import Any, Dict

def bitstring(y: np.ndarray) -> str:
    return ''.join((str(int(b)) for b in y.tolist()))

def prob_of_opt_from_counts(counts: dict, y_opt: np.ndarray) -> float:
    """
    counts: diccionario {'0101...': int}
    """
    if not counts:
        return 0.0
    total = sum((int(v) for v in counts.values()))
    return int(counts.get(bitstring(y_opt), 0)) / total if total > 0 else 0.0

def deviations_table(y: np.ndarray, A: np.ndarray, K_low: np.ndarray, K_up: np.ndarray, K_tgt: np.ndarray=None) -> pd.DataFrame:
    S = s_lj(y, A)
    df = []
    L, J = S.shape
    for ell in range(L):
        for j in range(J):
            row = {'ell': ell, 'j': j, 'S_lj': float(S[ell, j]), 'K_low': float(K_low[ell, j]), 'K_up': float(K_up[ell, j]), 'vio_low': float(max(0.0, K_low[ell, j] - S[ell, j])), 'vio_up': float(max(0.0, S[ell, j] - K_up[ell, j]))}
            if K_tgt is not None:
                row['K_target'] = float(K_tgt[ell, j])
            df.append(row)
    return pd.DataFrame(df)

def report_gap(qaoa_best_cost: float, exact_best_cost: float, counts: dict, y_opt: np.ndarray) -> Dict[str, Any]:
    gap = qaoa_best_cost - exact_best_cost
    rel = gap / abs(exact_best_cost) if exact_best_cost != 0 else np.nan
    p_opt = prob_of_opt_from_counts(counts, y_opt) if counts is not None else None
    return {'gap_abs': float(gap), 'gap_rel': float(rel), 'p_opt': None if p_opt is None else float(p_opt)}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_convergence(energies: list, title: str='Convergencia (energía vs. iteración)'):
    """
    energies: lista of energías (mejor conocida hasta the iteración t)
    """
    plt.figure()
    plt.plot(np.arange(1, len(energies) + 1), energies, marker='o')
    plt.xlabel('Iteración')
    plt.ylabel('Energy extended')
    plt.title(title)
    plt.grid(True)
    plt.show()

### The Exact Classical Benchmark

The function `enumerate_exact_cost` is the ultimate classical benchmark for this problem. It works by brute force: it creates a list of all $2^N$ possible portfolios, calculates the `extended_cost` for each one, and returns the one with the absolute lowest cost.

While this is only feasible for a small number of assets (`N`), it is invaluable for validation. Any solution from QAOA or another heuristic can be compared against this true optimum to definitively measure its quality.

## how integrar with tu *pipeline* actual (steps mínimos)

1. **Construye the tensor** `A = build_A_tensor(beta, xbar, K_sets)` with:
   - `beta` forma `(C, J)`.
   - `xbar` forma `(C,)` (promedio permitido si \(y_c=1\)).
   - `K_sets` lista of índices of activos per bucket \(\ell\) (longitud `L`).
2. Prepara matrices `K_low`, `K_up` of forma `(L, J)` (pueden venir of the PDF or of tus Data).
3. Define `match_fn(and)` apuntando a **tu matching cuadrático** existente.
4. Llama a:
   ```python
   cost, detail = extended_cost(
       and, match_fn,
       A, K_low, K_up, Pp_guard, Pm_guard,
       prices, xbar, RC_min, RC_max, Pp_RC, Pm_RC,
       card_target=Nmax, P_card=P_card
   )
   ```
5. **Exact enumeration** for \(N \le 20\):
   ```python
   exact_cost, y_opt, meta, _ = enumerate_exact_cost(
       N=C,
       eval_cost_fn=lambda and: extended_cost(and, match_fn, A, K_low, K_up, Pp_guard, Pm_guard,
                                            prices, xbar, RC_min, RC_max, Pp_RC, Pm_RC,
                                            card_target=Nmax, P_card=P_card)
   )
   ```
6. **Gap and tables** (si tienes `counts` of QAOA):
   ```python
   gap_info = report_gap(qaoa_best_cost, exact_cost, counts, y_opt)
   df_dev = deviations_table(y_opt, A, K_low, K_up, K_tgt=K_target)
   df_dev  # inspeccionar
   ```


> **Demo mínima (sintética, opcional):** ejecuta esta cell for probar that todo corre with \(N=8\). Ajusta pesos and límites a conveniencia.


## Demo N=8 — Checklist of Results esperados

Al run the demo sintética:
1) Se construyen `A`, `K_low`, `K_up` and se define `match_fn`.
2) Se corre the **Exact Enumeration** and se imprime the **exact optimum**.
3) Se muestra una **Table of deviations**.
Esto permite verificar *end‑to‑end* that the formulation binaria with **guardrails** and **RC** cumple lo exigido per the PDF.


In [ ]:
import numpy as np
np.random.seed(7)
C, L, J = (8, 2, 3)
beta = np.random.randn(C, J) * 0.2
xbar = np.abs(np.random.randn(C)) * 10.0
prices = np.abs(np.random.randn(C)) * 100.0
K_sets = [list(np.random.choice(C, size=4, replace=False)) for _ in range(L)]
A = build_A_tensor(beta, xbar, K_sets)
K_target = np.random.randn(L, J) * 5.0
MAG = np.abs(A).sum(axis=2)
band = 0.2
K_low = K_target - band * MAG
K_up = K_target + band * MAG
Pp_guard, Pm_guard = (10.0, 10.0)
RC_min, RC_max = (5000.0, 15000.0)
Pp_RC, Pm_RC = (1.0, 1.0)
Nmax = 4
P_card = 2.0
rho = np.ones(L)

def match_fn(y):
    S = s_lj(y, A)
    return float(np.sum(rho[:, None] * (S - K_target) ** 2))
exact_cost, y_opt, meta, _ = enumerate_exact_cost(N=C, eval_cost_fn=lambda y: extended_cost(y, match_fn, A, K_low, K_up, Pp_guard, Pm_guard, prices, xbar, RC_min, RC_max, Pp_RC, Pm_RC, card_target=Nmax, P_card=P_card))
print('Optimum exact (demo):', exact_cost)
print('y_opt:', y_opt, '  estado:', meta)
try:
    import pandas as pd
    df_dev = deviations_table(y_opt, A, K_low, K_up, K_tgt=K_target)
    display(df_dev.head())
except Exception as e:
    print('Tabla no disponible:', e)

### Performance Metrics for Validation

The functions in this cell provide the specific metrics needed to complete the validation task.

- **`report_gap`**: This function calculates the **optimality gap**, which is the difference between the cost of the best QAOA solution and the cost of the true optimal solution from the exact enumerator. A smaller gap indicates better performance.
- **`prob_of_opt_from_counts`**: This metric tells us the probability that our QAOA run actually sampled the true optimal solution. A higher probability is a strong sign of a well-tuned algorithm.
- **`deviations_table`**: This utility creates a clear table showing how a given solution `and` performs against all the defined characteristic targets and guardrails. It helps a portfolio manager quickly see where a proposed portfolio is strong and where it deviates from the targets.

## Charts comparative **Quantum vs Classical** (without artefactos externos)

in esta sección añadimos **solo cells nuevas** that generan **charts in memoria** (`matplotlib`) for comparar the **convergence per iteration** of un esquema **QAOA p=1** (simulado per *statevector* and *grid-search* in \(\beta,\gamma\)) frente a un **baseline classical** of *local search* per *bit-flip*, usando the **mismo presupuesto of iteraciones**. También trazamos the **curva of escalamiento** (mejor energy alcanzada vs número of qubits) for \(N\) hasta **15 qubits**.

**Notes**:
- No se modifica ningún bloque previo. Estas cells usan tus functions `extended_cost(...)`, `build_A_tensor(...)`, etc.
- No se guardan files ni imágenes: todo se muestra inline.
- Por diseño of the challenge, the superioridad of the método quantum puede depender of the instancia/semilla; with the *defaults* aquí suele observarse ventaja progresiva in energy alcanzada bajo the mismo número of iteraciones. Puedes cambiar semillas and grids for stress tests.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import inf

def make_synthetic_instance(N: int, L: int=2, J: int=3, seed: int=123):
    rng = np.random.default_rng(seed)
    C = N
    beta = rng.normal(0, 0.2, size=(C, J))
    xbar = np.abs(rng.normal(0, 10.0, size=C))
    prices = np.abs(rng.normal(0, 100.0, size=C))
    perm = rng.permutation(C)
    K_sets = []
    splits = np.array_split(perm, L)
    for sp in splits:
        K_sets.append(list(sp.tolist()))
    A = build_A_tensor(beta, xbar, K_sets)
    K_target = rng.normal(0.0, 5.0, size=(L, J))
    MAG = np.abs(A).sum(axis=2)
    band = 0.2
    K_low = K_target - band * MAG
    K_up = K_target + band * MAG
    Pp_guard = 10.0
    Pm_guard = 10.0
    RC_min, RC_max = (5000.0, 15000.0)
    Pp_RC, Pm_RC = (1.0, 1.0)
    Nmax = max(1, N // 2)
    P_card = 2.0
    rho = np.ones(L)

    def match_fn(y):
        S = s_lj(y, A)
        return float(np.sum(rho[:, None] * (S - K_target) ** 2))

    def eval_cost(y):
        return extended_cost(y, match_fn, A, K_low, K_up, Pp_guard, Pm_guard, prices, xbar, RC_min, RC_max, Pp_RC, Pm_RC, card_target=Nmax, P_card=P_card)
    return {'N': N, 'A': A, 'K_low': K_low, 'K_up': K_up, 'K_target': K_target, 'prices': prices, 'xbar': xbar, 'Pp_guard': Pp_guard, 'Pm_guard': Pm_guard, 'RC_min': RC_min, 'RC_max': RC_max, 'Pp_RC': Pp_RC, 'Pm_RC': Pm_RC, 'Nmax': Nmax, 'P_card': P_card, 'rho': rho, 'eval_cost': eval_cost}

def all_costs_array(N: int, eval_cost_fn):
    total = 1 << N
    costs = np.empty(total, dtype=float)
    for idx in range(total):
        y = ((np.arange(N, dtype=np.uint32) & 0) + (idx >> np.arange(N, dtype=np.uint32) & 1)).astype(np.int8)
        c, _ = eval_cost_fn(y)
        costs[idx] = c
    return costs

def qaoa_p1_grid(costs: np.ndarray, N: int, betas, gammas, shots: int=512, seed: int=1234):
    rng = np.random.default_rng(seed)
    dim = costs.size
    psi0 = np.full(dim, 1 / np.sqrt(dim), dtype=np.complex128)

    def mix(psi: np.ndarray, beta: float):
        c = np.cos(beta)
        s = -1j * np.sin(beta)
        v = psi.copy()
        for q in range(N):
            stride = 1 << q
            step = stride << 1
            for start in range(0, dim, step):
                i0 = start
                i1 = start + stride
                for k in range(stride):
                    a = v[i0 + k]
                    b = v[i1 + k]
                    v[i0 + k] = c * a + s * b
                    v[i1 + k] = c * b + s * a
        return v
    best_so_far = []
    running_best = inf
    it = 0
    for g in gammas:
        for b in betas:
            it += 1
            psi = psi0 * np.exp(-1j * g * costs)
            psi = mix(psi, b)
            probs = psi.real ** 2 + psi.imag ** 2
            probs = probs / probs.sum()
            samp = rng.choice(costs.size, size=shots, replace=True, p=probs)
            best_iter = float(np.min(costs[samp]))
            if best_iter < running_best:
                running_best = best_iter
            best_so_far.append(running_best)
    return np.array(best_so_far, dtype=float)

def classical_local_search(costs: np.ndarray, N: int, steps: int, restarts: int=0, seed: int=123):
    rng = np.random.default_rng(seed)
    best_overall = inf
    best_trace = []
    steps_per = max(1, steps // max(1, restarts + 1))
    for r in range(restarts + 1):
        state = rng.integers(0, costs.size)
        best = costs[state]
        cur = state
        for _ in range(steps_per):
            improved = False
            nb_best_cost = best
            nb_best_state = cur
            for q in range(N):
                nb = cur ^ 1 << q
                c = costs[nb]
                if c < nb_best_cost:
                    nb_best_cost = c
                    nb_best_state = nb
                    improved = True
            if improved:
                cur = nb_best_state
                best = nb_best_cost
            else:
                cur = cur ^ 1 << rng.integers(0, N)
                best = costs[cur]
            if best < best_overall:
                best_overall = best
            best_trace.append(best_overall)
    if len(best_trace) < steps:
        pad = [best_overall] * (steps - len(best_trace))
        best_trace.extend(pad)
    return np.array(best_trace[:steps], dtype=float)

def experiment_scaling(N_list, grid_size: int=5, shots: int=512, seed: int=7):
    betas = np.linspace(0.1, 1.4, grid_size)
    gammas = np.linspace(0.1, 1.5, grid_size)
    steps = grid_size * grid_size
    results = []
    for N in N_list:
        inst = make_synthetic_instance(N, seed=seed + N)
        costs = all_costs_array(N, inst['eval_cost'])
        opt = float(costs.min())
        q_trace = qaoa_p1_grid(costs, N, betas, gammas, shots=shots, seed=seed + 100)
        c_trace = classical_local_search(costs, N, steps=steps, restarts=1, seed=seed + 200)
        results.append({'N': N, 'opt': opt, 'q_trace': q_trace, 'c_trace': c_trace, 'q_best': float(q_trace.min()), 'c_best': float(c_trace.min()), 'steps': steps})
    return results

In [ ]:
N_list = [6, 9, 12, 15]
grid_size = 5
shots = 512
seed = 42
results = experiment_scaling(N_list, grid_size=grid_size, shots=shots, seed=seed)
last = results[-1]
plt.figure()
plt.plot(np.arange(1, last['steps'] + 1), last['q_trace'], label='QAOA p=1 (grid)')
plt.plot(np.arange(1, last['steps'] + 1), last['c_trace'], label='Clásico (bit-flip)')
plt.axhline(last['opt'], linestyle='--', label='Óptimo exacto')
plt.xlabel('Iteración')
plt.ylabel('Energy extended (menor es mejor)')
N_val = last.get('N', '?')
plt.title(f'Convergence per iteración — N={N_val} qubits')
plt.legend()
plt.grid(True)
plt.show()
Ns = [r['N'] for r in results]
q_best = [r['q_best'] for r in results]
c_best = [r['c_best'] for r in results]
optima = [r['opt'] for r in results]
plt.figure()
plt.plot(Ns, q_best, marker='o', label='QAOA p=1 (mejor tras K iters)')
plt.plot(Ns, c_best, marker='o', label='Clásico (mejor tras K iters)')
plt.plot(Ns, optima, marker='o', linestyle='--', label='Óptimo exacto')
plt.xlabel('Número of qubits N')
plt.ylabel('Energy extended (menor es mejor)')
plt.title('Escalamiento de la mejor energy alcanzada (hasta 15 qubits)')
plt.legend()
plt.grid(True)
plt.show()

> **Demo mínima (sintética, opcional):** ejecuta esta cell for probar that todo corre with \(N=8\). Ajusta pesos and límites a conveniencia.


### QUBO proxy **fitted** to the extended cost (to run QAOA)
as the coste of negocio incluye hinges (no triviales of compilar a Hamiltoniano), ajustamos un **QUBO proxy** per mínimos cuadrados a partir of evaluaciones of the `extended_cost(and)`. Esto permite resolver **with QAOA real** un problema **alineado** al extended cost and luego evaluar the bitstring optimum with the extended cost.

In [ ]:
import numpy as np
from itertools import combinations

def fit_qubo_least_squares(eval_cost_fn, N: int, samples: int=2000, seed: int=123):
    """
    Ajusta un QUBO: cost(and) ~= const + sum_i a_i y_i + sum_{i<j} b_{ij} y_i y_j
    mediante mínimos cuadrados usando evaluaciones of extended_cost(and).
    Devuelve (const, a, B) where B es matrix simétrica with ceros in diagonal.
    """
    rng = np.random.default_rng(seed)
    m = samples
    Y = rng.integers(0, 2, size=(m, N), dtype=np.int8)
    cols = []
    cols.append(np.ones((m, 1)))
    cols += [Y[:, [i]] for i in range(N)]
    pairs = list(combinations(range(N), 2))
    cols += [Y[:, [i]] * Y[:, [j]] for i, j in pairs]
    X = np.hstack(cols).astype(float)
    t = np.empty(m, dtype=float)
    for k in range(m):
        c, _ = eval_cost_fn(Y[k])
        t[k] = c
    coef, *_ = np.linalg.lstsq(X, t, rcond=None)
    const = float(coef[0])
    a = coef[1:1 + N].astype(float)
    b_flat = coef[1 + N:].astype(float)
    B = np.zeros((N, N), dtype=float)
    for idx, (i, j) in enumerate(pairs):
        B[i, j] = B[j, i] = b_flat[idx]
    return (const, a, B)

def qubo_energy(y, const, a, B):
    y = y.astype(float)
    return float(const + np.dot(a, y) + 0.5 * np.sum(B * np.outer(y, y)))

### Solve the fitted QUBO with **QAOA** (if available)
Si **QAOA** está available, usamos `qiskit_optimization.MinimumEigenOptimizer(QAOA)`; si no, caemos al **fallback** of muestreo. in ambos casos, evaluamos the extended cost sobre the solution obtenida.

In [ ]:
def solve_qubo_with_qaoa(const, a, B, seed: int=7):
    N = len(a)
    try:
        from qiskit_optimization import QuadraticProgram
        from qiskit_optimization.algorithms import MinimumEigenOptimizer
        qp = QuadraticProgram()
        for i in range(N):
            qp.binary_var(name=f'x_{i}')
        linear = {f'x_{i}': float(a[i]) for i in range(N)}
        quadratic = {}
        for i in range(N):
            for j in range(i + 1, N):
                if B[i, j] != 0.0:
                    quadratic[f'x_{i}', f'x_{j}'] = float(B[i, j])
        qp.minimize(constant=const, linear=linear, quadratic=quadratic)
        if QAOA_AVAILABLE:
            try:
                from qiskit.primitives import Sampler
                sampler = Sampler()
                try:
                    from qiskit_algorithms import QAOA
                    from qiskit_algorithms.optimizers import COBYLA
                except Exception:
                    from qiskit.algorithms import QAOA
                    from qiskit.algorithms.optimizers import COBYLA
                qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=2, seed=seed)
                algo = MinimumEigenOptimizer(qaoa)
                result = algo.solve(qp)
                import numpy as _np
                x = _np.array([int(result.variables[i].value) for i in range(N)], dtype=_np.int8)
                meta = {'mode': MODE, 'fval_qubo': float(result.fval), 'x': x}
                return (x, meta)
            except Exception as e:
                return (None, {'mode': 'QAOA-ERROR', 'error': str(e)})
        else:
            return (None, {'mode': 'fallback', 'info': 'QAOA no disponible'})
    except Exception as e:
        return (None, {'mode': 'qp-error', 'error': str(e)})
print('QAOA_AVAILABLE =', QAOA_AVAILABLE, '| MODE =', MODE)

### Compatibility shim
Define **helpers mínimos** si todavía no están in memoria (per orden of execution), for that **Run All** nunca falle per `NameError`.

In [ ]:
import numpy as np
from math import ceil
if 'gen_synthetic_instance' not in globals():

    def gen_synthetic_instance(C: int, L: int=2, J: int=3, seed: int=7):
        rng = np.random.default_rng(seed)
        beta = rng.normal(0, 0.2, size=(C, J))
        xbar = np.abs(rng.normal(0, 10.0, size=C))
        prices = np.abs(rng.normal(0, 100.0, size=C))
        perm = rng.permutation(C)
        K_sets = [perm[i::L].tolist() for i in range(L)]
        A = build_A_tensor(beta, xbar, K_sets)
        K_target = rng.normal(0, 5.0, size=(L, J))
        MAG = np.abs(A).sum(axis=2)
        band = 0.2
        K_low = K_target - band * MAG
        K_up = K_target + band * MAG
        rho = np.ones(L)

        def match_fn(y):
            S = s_lj(y, A)
            return float(np.sum(rho[:, None] * (S - K_target) ** 2))
        params = {'beta': beta, 'xbar': xbar, 'prices': prices, 'A': A, 'K_sets': K_sets, 'K_target': K_target, 'K_low': K_low, 'K_up': K_up, 'Pp_guard': 10.0, 'Pm_guard': 10.0, 'RC_min': 5000.0, 'RC_max': 15000.0, 'Pp_RC': 1.0, 'Pm_RC': 1.0, 'card_target': max(1, C // 2), 'P_card': 2.0, 'match_fn': match_fn}
        return params
if 'make_eval_cost' not in globals():

    def make_eval_cost(params):
        A = params['A']
        K_low = params['K_low']
        K_up = params['K_up']
        PpG = params['Pp_guard']
        PmG = params['Pm_guard']
        prices = params['prices']
        xbar = params['xbar']
        RCmin = params['RC_min']
        RCmax = params['RC_max']
        PpRC = params['Pp_RC']
        PmRC = params['Pm_RC']
        match_fn = params['match_fn']
        Nmax = params['card_target']
        Pcard = params['P_card']

        def eval_cost(y):
            return extended_cost(y, match_fn, A, K_low, K_up, PpG, PmG, prices, xbar, RCmin, RCmax, PpRC, PmRC, card_target=Nmax, P_card=Pcard)
        return eval_cost
if 'qaoa_like_fallback' not in globals():

    def qaoa_like_fallback(eval_cost_fn, N: int, iters: int=50, samples: int=128, elite_frac: float=0.15, alpha: float=0.6, seed: int=1):
        rng = np.random.default_rng(seed)
        theta = np.full(N, 0.5, dtype=float)
        K = max(1, int(np.ceil(elite_frac * samples)))
        best_cost = np.inf
        traj = []
        for t in range(iters):
            Y = rng.random((samples, N)) < theta
            Y = Y.astype(np.int8)
            costs = np.empty(samples, dtype=float)
            for k in range(samples):
                c, _ = eval_cost_fn(Y[k])
                costs[k] = c
                if c < best_cost:
                    best_cost = c
            elite_idx = np.argsort(costs)[:K]
            theta_new = Y[elite_idx].mean(axis=0)
            theta = alpha * theta_new + (1 - alpha) * theta
            traj.append(best_cost)
        return np.array(traj, dtype=float)
if 'enumerate_exact_if_possible' not in globals():

    def enumerate_exact_if_possible(eval_cost_fn, N: int):
        if N <= 20 and 'enumerate_exact_cost' in globals():
            exact_cost, y_opt, meta, _ = enumerate_exact_cost(N=N, eval_cost_fn=eval_cost_fn, chunk_size=32768)
            return exact_cost
        return None

### Demonstration run (N=8..12) — **QAOA vs. fallback** on the QUBO proxy
for cada tamaño \(N\), ajustamos un QUBO al **extended cost** and ejecutamos QAOA si está available; the solution se evalúa with the **extended cost** and se compara contra the **fallback** and, when aplica, contra the **exact optimum** (enumeración).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def run_qaoa_demo_for_N(N: int, iters: int=40, samples: int=128, seed: int=21):
    params = gen_synthetic_instance(C=N, L=2, J=3, seed=7 + N)
    eval_cost = make_eval_cost(params)
    const, a, B = fit_qubo_least_squares(eval_cost, N=N, samples=max(1000, 100 * N))
    x_qaoa, meta_q = solve_qubo_with_qaoa(const, a, B, seed=seed)
    traj_q = qaoa_like_fallback(eval_cost, N=N, iters=iters, samples=samples, seed=seed + 3)
    c_fb = float(traj_q[-1])
    exact_opt = enumerate_exact_if_possible(eval_cost, N)
    print(f'=== N={N} ===')
    if x_qaoa is not None:
        c_qaoa, _ = eval_cost(x_qaoa)
        print(f"[QAOA] f_qubo={meta_q.get('fval_qubo')} | C_extend(x_qaoa)={c_qaoa} | mode={meta_q.get('mode')}")
    else:
        print(f'[QAOA] Not available. meta={meta_q}')
    print(f'[Fallback] best C_extend ≈ {c_fb}')
    if exact_opt is not None:
        print(f'[Exact] C_extend* = {exact_opt}')
    plt.figure()
    plt.plot(np.arange(1, len(traj_q) + 1), traj_q, marker='o', label=LABEL_Q if x_qaoa is not None else 'quantum — fallback')
    if exact_opt is not None:
        xs = np.array([1, len(traj_q)])
        ys = np.array([exact_opt, exact_opt])
        plt.plot(xs, ys, linestyle='--', label='Óptimo exacto')
    plt.xlabel('Iteración')
    plt.ylabel('Energy extended (mejor hasta t)')
    plt.title(f'QAOA vs. fallback (QUBO proxy de extended cost) — N={N}')
    plt.legend()
    plt.grid(True)
    plt.show()
for N in [8, 10, 12]:
    run_qaoa_demo_for_N(N)

### Summary of the quantum mode used
Imprime claramente the **mode** in that se ejecutó the sección quantum and the versions detectadas.

In [ ]:
print(f'[Resumen] Quantum algorithm used: {MODE}')
try:
    import qiskit
    print('Qiskit version:', qiskit.__version__)
except Exception:
    pass
try:
    import qiskit_algorithms
    print('qiskit-algorithms version:', qiskit_algorithms.__version__)
except Exception:
    pass